In [6]:
import requests
import json
import pandas as pd
import time
from tqdm import tqdm

In [7]:
def get_pushshift_data(data_type, **kwargs):
    """
    Gets data from the pushshift api.
 
    data_type can be 'comment' or 'submission'
    The rest of the args are interpreted as payload.
 
    Read more: https://github.com/pushshift/api
    
    Credit: https://www.jcchouinard.com/how-to-use-reddit-api-with-python/
    """
 
    base_url = f"https://api.pushshift.io/reddit/search/{data_type}/"
    payload = kwargs
    request = requests.get(base_url, params=payload)
    return request.json()

In [8]:
def extract_title(x):
    try:
        title = x['title']
        return title
    except:
        return 0
    
def extract_text(x):
    try:
        text = x['selftext']
        return text
    except:
        return 0

In [9]:
# specify posts from [1st-8th) and [15-22) of every month (every other week)
dates = {
         'Feb1': ['1580515200', '1581120000'],
         'Feb2': ['1581724800', '1582329600'],
         'Mar1': ['1583020800', '1583625600'],
         'Mar2': ['1584230400', '1584835200'],
         'Apr1': ['1585699200', '1586304000'],
         'Apr2': ['1586908800', '1587513600'],
         'May1': ['1588291200', '1588896000'],
         'May2': ['1589500800', '1590105600'],
         'Jun1': ['1590969600', '1591574400'],
         'Jun2': ['1592179200', '1592784000'],
         'Jul1': ['1593561600', '1594166400'],
         'Jul2': ['1594771200', '1595376000'],
         'Aug1': ['1596240000', '1596844800'],
         'Aug2': ['1597449600', '1598054400'],
         'Sep1': ['1598918400', '1599523200'],
         'Sep2': ['1600128000', '1600732800'],
         'Oct1': ['1601510400', '1602115200'],
         'Oct2': ['1602720000', '1603324800'],
         'Nov1': ['1604188800', '1604793600'],
         'Nov2': ['1605398400', '1606003200'],
         'Dec1': ['1606780800', '1607385600'],
         'Dec2': ['1607990400', '1608595200'],
         'Jan1': ['1609459200', '1610064000'],
         'Jan2': ['1610668800', '1611273600']
        }

In [ ]:
for week in tqdm(dates.keys()):
    print(f'SCRAPING {week} =====================================')
    begin, end = dates[week]
    success = False
    while not success:
        try:
            data = get_pushshift_data(data_type='submission',
                                          before=end,
                                          after=begin,
                                          subreddit='MentalHealth',
                                          sort_type='created_utc',
                                          sort='asc',
                                          size=1000)
            df = pd.DataFrame.from_dict(data)
            success = True
        except:
            print('exception encountered...continuing')
            time.sleep(1)
            continue
    
    timestamp = str(df.data.iloc[-1]['created_utc'])
    while timestamp < end:
        try:
            data = get_pushshift_data(data_type='submission',
                                      before=end,
                                      after=timestamp,
                                      subreddit='MentalHealth',
                                      sort_type='created_utc',
                                      sort='asc',
                                      size=1000)
            df_new = df.append(pd.DataFrame.from_dict(data))
            if df_new.data.iloc[-1]['created_utc'] == df.data.iloc[-1]['created_utc']:
                break
            else:
                df = df_new
                timestamp = str(df.data.iloc[-1]['created_utc'])
                time.sleep(1)

        except:
            time.sleep(1)
            continue

    title_df = df.data.apply(extract_title).reset_index()['data'].rename('title')
    text_df = df.data.apply(extract_text).reset_index()['data'].rename('text')
    full_df = pd.concat([title_df, text_df], axis=1)
    full_df.to_csv(f'raw_data/{week}.csv')

  0%|          | 0/24 [00:00<?, ?it/s]

SCRAPING Feb1 =====================================
1580578465


  4%|▍         | 1/24 [00:40<15:35, 40.67s/it]

SCRAPING Feb2 =====================================
1581764085


  8%|▊         | 2/24 [01:38<18:40, 50.95s/it]

...

SCRAPING Nov2 =====================================
1605437639


 83%|████████▎ | 20/24 [17:18<03:34, 53.74s/it]

SCRAPING Dec1 =====================================
1606808699


 88%|████████▊ | 21/24 [18:01<02:30, 50.33s/it]

SCRAPING Dec2 =====================================
1608029906
